In [32]:
import os
import torch
from ultralytics import YOLO
import gc

def clear_memory():
    """Clear GPU memory"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

def optimize_gpu_memory():
    """Optimize GPU memory settings"""
    if torch.cuda.is_available():
        # Set memory allocation strategy
        os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
        
        # Clear cache
        torch.cuda.empty_cache()
        
        # Get GPU info
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"🖥️  GPU: {torch.cuda.get_device_name(0)}")
        print(f"💾 Total GPU Memory: {gpu_memory:.1f} GB")
        
        # Calculate optimal batch size for different GPU memory sizes
        if gpu_memory <= 4:
            recommended_batch = 2
            recommended_workers = 2
        elif gpu_memory <= 6:
            recommended_batch = 4
            recommended_workers = 4
        elif gpu_memory <= 8:
            recommended_batch = 8
            recommended_workers = 6
        else:
            recommended_batch = 16
            recommended_workers = 8
            
        return recommended_batch, recommended_workers
    else:
        return 4, 2  # CPU fallback

def train_memory_optimized():
    print("🔴 Memory-Optimized YOLOv8 Red License Plate Training")
    print("=" * 60)
    
    # Clear memory first
    clear_memory()
    
    # Optimize memory settings
    batch_size, workers = optimize_gpu_memory()
    
    # Dataset path
    dataset_path = "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset"
    
    # Create models directory in dataset path
    models_dir = os.path.join(dataset_path, "models")
    os.makedirs(models_dir, exist_ok=True)
    
    # Use smaller model for limited GPU memory
    print("🤖 Using YOLOv8n (Nano) for memory efficiency...")
    model = YOLO('yolov8n.pt')
    
    # Memory-optimized training configuration
    train_config = {
        'data': f'{dataset_path}/data.yaml',
        'epochs': 100,
        'imgsz': 416,
        'batch': batch_size,
        'name': 'red_plate_memory_opt',
        'patience': 30,
        'save': True,
        'plots': True,
        'device': 0 if torch.cuda.is_available() else 'cpu',
        'workers': workers,
        'project': models_dir,  # Changed to save in dataset/models
        'exist_ok': True,
        
        # Memory optimization
        'amp': True,
        'fraction': 0.9,
        'cache': False,
        'rect': True,
        'resume': False,
        'single_cls': False,
        
        # Learning rate parameters
        'lr0': 0.005,
        'lrf': 0.1,
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'warmup_epochs': 2,
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.1,
        
        # Loss weights
        'box': 7.5,
        'cls': 0.5,
        'dfl': 1.5,
        
        # Data augmentation parameters
        'hsv_h': 0.01,
        'hsv_s': 0.5,
        'hsv_v': 0.3,
        'degrees': 0.0,
        'translate': 0.05,
        'scale': 0.3,
        'shear': 0.0,
        'perspective': 0.0,
        'flipud': 0.0,
        'fliplr': 0.5,
        'mosaic': 0.5,
        'mixup': 0.0,
        'copy_paste': 0.0
    }
    
    print("📋 Memory-Optimized Configuration:")
    print(f"   Image size: {train_config['imgsz']}")
    print(f"   Batch size: {train_config['batch']}")
    print(f"   Workers: {train_config['workers']}")
    print(f"   AMP enabled: {train_config['amp']}")
    print(f"   Models will be saved to: {models_dir}")
    print("=" * 60)
    
    try:
        # Clear memory before training
        clear_memory()
        
        # Start training
        print("🚀 Starting memory-optimized training...")
        results = model.train(**train_config)
        
        print("🎉 Training completed successfully!")
        
        # Get the actual save directory
        save_dir = os.path.join(models_dir, 'red_plate_memory_opt')
        print(f"💾 Best model saved at: {save_dir}/weights/best.pt")
        
        # Clear memory after training
        clear_memory()
        
        # Quick validation
        print("\n🔍 Running validation...")
        metrics = model.val()
        
        print("📊 Validation Results:")
        if hasattr(metrics, 'box'):
            print(f"   mAP50: {metrics.box.map50:.4f}")
            print(f"   mAP50-95: {metrics.box.map:.4f}")
            print(f"   Precision: {metrics.box.mp:.4f}")
            print(f"   Recall: {metrics.box.mr:.4f}")
        else:
            print("   Validation metrics computed successfully")
        
        return save_dir
        
    except torch.cuda.OutOfMemoryError:
        print("❌ Still out of memory! Trying CPU training...")
        clear_memory()
        return train_on_cpu()
    except Exception as e:
        print(f"❌ Training failed: {str(e)}")
        return None

def train_on_cpu():
    """Fallback to CPU training"""
    print("🖥️  Fallback: Training on CPU...")
    
    dataset_path = "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset"
    
    # Create models directory in dataset path
    models_dir = os.path.join(dataset_path, "models")
    os.makedirs(models_dir, exist_ok=True)
    
    model = YOLO('yolov8n.pt')
    
    # CPU-optimized configuration
    cpu_config = {
        'data': f'{dataset_path}/data.yaml',
        'epochs': 50,
        'imgsz': 320,
        'batch': 4,
        'name': 'red_plate_cpu',
        'patience': 20,
        'save': True,
        'plots': True,
        'device': 'cpu',
        'workers': 2,
        'project': models_dir,  # Changed to save in dataset/models
        'exist_ok': True,
        
        # Minimal augmentation
        'hsv_h': 0.01,
        'hsv_s': 0.3,
        'hsv_v': 0.2,
        'degrees': 0.0,
        'translate': 0.02,
        'scale': 0.2,
        'fliplr': 0.5,
        'mosaic': 0.3,
        'mixup': 0.0,
        'copy_paste': 0.0
    }
    
    try:
        results = model.train(**cpu_config)
        print("🎉 CPU training completed!")
        
        # Get save directory
        save_dir = os.path.join(models_dir, 'red_plate_cpu')
        return save_dir
        
    except Exception as e:
        print(f"❌ CPU training also failed: {str(e)}")
        return None

def ultra_minimal_training():
    """Ultra minimal config for very limited resources"""
    print("🔬 Ultra Minimal Training for Limited Resources")
    
    dataset_path = "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset"
    
    # Create models directory in dataset path
    models_dir = os.path.join(dataset_path, "models")
    os.makedirs(models_dir, exist_ok=True)
    
    # Clear all memory
    clear_memory()
    
    model = YOLO('yolov8n.pt')
    
    minimal_config = {
        'data': f'{dataset_path}/data.yaml',
        'epochs': 30,
        'imgsz': 256,
        'batch': 1,
        'name': 'red_plate_minimal',
        'patience': 15,
        'save': True,
        'plots': False,
        'device': 0 if torch.cuda.is_available() else 'cpu',
        'workers': 0,
        'project': models_dir,  # Changed to save in dataset/models
        'exist_ok': True,
        'cache': False,
        'amp': True,
        'fraction': 0.8,
        
        # No augmentation
        'hsv_h': 0.0,
        'hsv_s': 0.0,
        'hsv_v': 0.0,
        'degrees': 0.0,
        'translate': 0.0,
        'scale': 0.0,
        'fliplr': 0.0,
        'mosaic': 0.0,
        'mixup': 0.0,
        'copy_paste': 0.0
    }
    
    print("⚡ Ultra minimal configuration:")
    print(f"   Image size: {minimal_config['imgsz']}")
    print(f"   Batch size: {minimal_config['batch']}")
    print(f"   Epochs: {minimal_config['epochs']}")
    print(f"   Models will be saved to: {models_dir}")
    
    try:
        results = model.train(**minimal_config)
        print("🎉 Ultra minimal training completed!")
        
        # Get save directory
        save_dir = os.path.join(models_dir, 'red_plate_minimal')
        return save_dir
        
    except Exception as e:
        print(f"❌ Even minimal training failed: {str(e)}")
        return None

def download_pretrained_models(dataset_path):
    """Download and save YOLO pretrained models to dataset directory"""
    models_dir = os.path.join(dataset_path, "pretrained_models")
    os.makedirs(models_dir, exist_ok=True)
    
    print("📥 Downloading pretrained models...")
    
    # List of YOLO models to download
    models = ['yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt']
    
    for model_name in models:
        try:
            print(f"   Downloading {model_name}...")
            model = YOLO(model_name)
            
            # Save model to our directory
            model_path = os.path.join(models_dir, model_name)
            torch.save(model.model.state_dict(), model_path.replace('.pt', '_weights.pt'))
            print(f"   ✅ {model_name} saved to {models_dir}")
            
        except Exception as e:
            print(f"   ❌ Failed to download {model_name}: {str(e)}")
    
    return models_dir

if __name__ == "__main__":
    # Set environment variable for memory optimization
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
    
    # Dataset path
    dataset_path = "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset"
    
    print("Choose training mode:")
    print("1. Memory Optimized (Recommended)")
    print("2. CPU Training")
    print("3. Ultra Minimal")
    print("4. Download Pretrained Models Only")
    
    choice = input("Enter choice (1/2/3/4) [default: 1]: ").strip()
    
    if choice == "4":
        models_dir = download_pretrained_models(dataset_path)
        print(f"\n✅ Pretrained models downloaded to: {models_dir}")
    elif choice == "2":
        save_dir = train_on_cpu()
    elif choice == "3":
        save_dir = ultra_minimal_training()
    else:
        save_dir = train_memory_optimized()
    
    if choice != "4":
        if save_dir:
            print(f"\n✅ Training successful! Model saved at: {save_dir}")
            print(f"📁 Best weights: {save_dir}/weights/best.pt")
            print(f"📁 Last weights: {save_dir}/weights/last.pt")
            print(f"📁 Training results: {save_dir}")
            
            # Show directory structure
            print(f"\n📂 Directory structure:")
            print(f"   {dataset_path}/")
            print(f"   ├── models/")
            print(f"   │   └── {os.path.basename(save_dir)}/")
            print(f"   │       ├── weights/")
            print(f"   │       │   ├── best.pt")
            print(f"   │       │   └── last.pt")
            print(f"   │       ├── results.png")
            print(f"   │       └── confusion_matrix.png")
            print(f"   └── data.yaml")
            
        else:
            print("\n❌ All training attempts failed. Consider:")
            print("   - Using a machine with more GPU memory")
            print("   - Training on Google Colab with GPU")
            print("   - Using cloud training services")

Choose training mode:
1. Memory Optimized (Recommended)
2. CPU Training
3. Ultra Minimal
4. Download Pretrained Models Only


Enter choice (1/2/3/4) [default: 1]:  1


🔴 Memory-Optimized YOLOv8 Red License Plate Training
🖥️  GPU: NVIDIA GeForce RTX 3050 Laptop GPU
💾 Total GPU Memory: 4.0 GB
🤖 Using YOLOv8n (Nano) for memory efficiency...


100%|█████████████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:00<00:00, 11.3MB/s]


📋 Memory-Optimized Configuration:
   Image size: 416
   Batch size: 2
   Workers: 2
   AMP enabled: True
   Models will be saved to: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset\models
🚀 Starting memory-optimized training...
Ultralytics 8.3.156  Python-3.12.4 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/minhk/Downloads/code thu/AI/NguyenLam/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=0.9, freeze=None, half=False, hsv_h=0.01, hsv_s=0.5, hsv_v=0.3, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005

...


96 epochs completed in 8.145 hours.
Optimizer stripped from C:\Users\minhk\Downloads\code thu\AI\NguyenLam\dataset\models\red_plate_memory_opt\weights\last.pt, 6.2MB
Optimizer stripped from C:\Users\minhk\Downloads\code thu\AI\NguyenLam\dataset\models\red_plate_memory_opt\weights\best.pt, 6.2MB

Validating C:\Users\minhk\Downloads\code thu\AI\NguyenLam\dataset\models\red_plate_memory_opt\weights\best.pt...
Ultralytics 8.3.156  Python-3.12.4 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 512/512 [00:


                   all       2048       2195      0.983      0.901      0.954      0.647
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to C:\Users\minhk\Downloads\code thu\AI\NguyenLam\dataset\models\red_plate_memory_opt
🎉 Training completed successfully!
💾 Best model saved at: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset\models\red_plate_memory_opt/weights/best.pt

🔍 Running validation...
Ultralytics 8.3.156  Python-3.12.4 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.20.1 ms, read: 34.813.6 MB/s, size: 20.2 KB)


val: Scanning C:\Users\minhk\Downloads\code thuê\AI\NguyenLam\dataset\valid\labels.cache... 2048 images, 3 backgrounds,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1024/1024 [0


                   all       2048       2195      0.984      0.901      0.955      0.647
Speed: 0.2ms preprocess, 5.3ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to C:\Users\minhk\Downloads\code thu\AI\NguyenLam\dataset\models\red_plate_memory_opt
📊 Validation Results:
   mAP50: 0.9548
   mAP50-95: 0.6471
   Precision: 0.9836
   Recall: 0.9007

✅ Training successful! Model saved at: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset\models\red_plate_memory_opt
📁 Best weights: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset\models\red_plate_memory_opt/weights/best.pt
📁 Last weights: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset\models\red_plate_memory_opt/weights/last.pt
📁 Training results: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset\models\red_plate_memory_opt

📂 Directory structure:
   C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset/
   ├── models/
   │   └── red_plate_memory_opt/
   │       ├── weights/
   │       │   

In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import cv2
from PIL import Image, ImageTk
import threading
import queue
import time
from ultralytics import YOLO
import numpy as np
import os
import easyocr
import re
from datetime import datetime

class LicensePlateOCRApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🔴 Red License Plate Detection & OCR")
        self.root.geometry("1400x900")
        self.root.configure(bg='#f0f0f0')
        
        # Initialize variables
        self.cap = None
        self.is_running = False
        self.frame_queue = queue.Queue(maxsize=2)
        self.confidence_threshold = 0.3
        self.current_frame = None
        self.detection_count = 0
        self.fps = 0
        self.detected_plates = []
        
        # OCR settings
        self.ocr_enabled = True
        self.ocr_reader = None
        
        # Load model and OCR
        self.load_model()
        self.load_ocr()
        
        # Setup GUI
        self.setup_gui()
        
        # Initialize webcam
        self.initialize_webcam()
    
    def load_model(self):
        """Load the trained YOLO model"""
        try:
            model_paths = [
                "runs/detect/red_plate_memory_opt/weights/best.pt",
                "runs/detect/red_plate_memory_opt/weights/last.pt",
                "runs/detect/red_plate_cpu/weights/best.pt",
                "runs/detect/red_plate_minimal/weights/best.pt"
            ]
            
            model_loaded = False
            for path in model_paths:
                if os.path.exists(path):
                    print(f"✅ Loading model from: {path}")
                    self.model = YOLO(path)
                    model_loaded = True
                    break
            
            if not model_loaded:
                print("⚠️  Using default YOLOv8n model")
                self.model = YOLO('yolov8n.pt')
            
            print(f"📊 Model classes: {self.model.names}")
            
        except Exception as e:
            print(f"❌ Error loading model: {e}")
            messagebox.showerror("Model Error", f"Cannot load model: {e}")
            self.model = None
    
    def load_ocr(self):
        """Load EasyOCR reader"""
        try:
            print("🔤 Loading OCR reader...")
            # Initialize EasyOCR with Vietnamese and English
            self.ocr_reader = easyocr.Reader(['vi', 'en'], gpu=True if cv2.cuda.getCudaEnabledDeviceCount() > 0 else False)
            print("✅ OCR reader loaded successfully")
        except Exception as e:
            print(f"❌ Error loading OCR: {e}")
            try:
                # Fallback without GPU
                self.ocr_reader = easyocr.Reader(['vi', 'en'], gpu=False)
                print("✅ OCR reader loaded (CPU mode)")
            except Exception as e2:
                print(f"❌ Cannot load OCR: {e2}")
                self.ocr_reader = None
                messagebox.showwarning("OCR Warning", "Cannot load OCR. Text recognition will be disabled.")
    
    def setup_gui(self):
        """Setup the enhanced GUI with OCR features"""
        # Main container
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Title
        title_frame = ttk.Frame(main_frame)
        title_frame.pack(fill=tk.X, pady=(0, 10))
        
        title_label = ttk.Label(title_frame, text="🔴 Red License Plate Detection & OCR System", 
                               font=('Arial', 18, 'bold'))
        title_label.pack()
        
        # Content frame
        content_frame = ttk.Frame(main_frame)
        content_frame.pack(fill=tk.BOTH, expand=True)
        
        # Left panel - Video display
        video_frame = ttk.LabelFrame(content_frame, text="📷 Live Camera Feed", padding="10")
        video_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=(0, 10))
        
        # Video display label
        self.video_label = ttk.Label(video_frame, text="📷 Initializing camera...", 
                                    background='black', foreground='white',
                                    font=('Arial', 12))
        self.video_label.pack(expand=True)
        
        # Right panel - Split into top and bottom
        right_panel = ttk.Frame(content_frame)
        right_panel.pack(side=tk.RIGHT, fill=tk.BOTH, padx=(10, 0))
        
        # Controls panel (top right)
        control_frame = ttk.LabelFrame(right_panel, text="🎛️ Controls", padding="10")
        control_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Confidence control
        conf_frame = ttk.Frame(control_frame)
        conf_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Label(conf_frame, text="🎯 Detection Confidence:", font=('Arial', 10, 'bold')).pack(anchor=tk.W)
        
        self.confidence_var = tk.DoubleVar(value=self.confidence_threshold)
        self.confidence_scale = ttk.Scale(conf_frame, from_=0.05, to=0.95, 
                                         variable=self.confidence_var, orient='horizontal', 
                                         length=200, command=self.update_confidence)
        self.confidence_scale.pack(fill=tk.X, pady=2)
        
        self.confidence_label = ttk.Label(conf_frame, text=f"Value: {self.confidence_threshold:.2f}")
        self.confidence_label.pack(anchor=tk.W)
        
        # OCR settings
        ocr_frame = ttk.Frame(control_frame)
        ocr_frame.pack(fill=tk.X, pady=(0, 10))
        
        self.ocr_var = tk.BooleanVar(value=self.ocr_enabled)
        ocr_check = ttk.Checkbutton(ocr_frame, text="🔤 Enable OCR Text Recognition", 
                                   variable=self.ocr_var, command=self.toggle_ocr)
        ocr_check.pack(anchor=tk.W)
        
        # Control buttons
        button_frame = ttk.Frame(control_frame)
        button_frame.pack(fill=tk.X, pady=(0, 10))
        
        self.start_button = ttk.Button(button_frame, text="▶️ Start Detection", 
                                      command=self.toggle_detection, width=20)
        self.start_button.pack(fill=tk.X, pady=1)
        
        self.screenshot_button = ttk.Button(button_frame, text="📸 Take Screenshot", 
                                           command=self.take_screenshot, width=20)
        self.screenshot_button.pack(fill=tk.X, pady=1)
        
        self.save_plates_button = ttk.Button(button_frame, text="💾 Save Detected Plates", 
                                            command=self.save_detected_plates, width=20)
        self.save_plates_button.pack(fill=tk.X, pady=1)
        
        self.clear_button = ttk.Button(button_frame, text="🗑️ Clear History", 
                                      command=self.clear_history, width=20)
        self.clear_button.pack(fill=tk.X, pady=1)
        
        self.reset_button = ttk.Button(button_frame, text="🔄 Reset Camera", 
                                      command=self.reset_camera, width=20)
        self.reset_button.pack(fill=tk.X, pady=1)
        
        self.quit_button = ttk.Button(button_frame, text="🚪 Quit", 
                                     command=self.quit_app, width=20)
        self.quit_button.pack(fill=tk.X, pady=1)
        
        # Information display
        info_frame = ttk.LabelFrame(right_panel, text="📊 Real-time Info", padding="10")
        info_frame.pack(fill=tk.X, pady=(0, 10))
        
        self.detection_var = tk.StringVar(value="Detections: 0")
        ttk.Label(info_frame, textvariable=self.detection_var, font=('Arial', 10)).pack(anchor=tk.W)
        
        self.fps_var = tk.StringVar(value="FPS: 0.0")
        ttk.Label(info_frame, textvariable=self.fps_var, font=('Arial', 10)).pack(anchor=tk.W)
        
        self.camera_var = tk.StringVar(value="Camera: Not connected")
        ttk.Label(info_frame, textvariable=self.camera_var, font=('Arial', 10)).pack(anchor=tk.W)
        
        self.model_var = tk.StringVar(value="Model: Loading...")
        ttk.Label(info_frame, textvariable=self.model_var, font=('Arial', 10)).pack(anchor=tk.W)
        
        self.ocr_status_var = tk.StringVar(value="OCR: Ready" if self.ocr_reader else "OCR: Not available")
        ttk.Label(info_frame, textvariable=self.ocr_status_var, font=('Arial', 10)).pack(anchor=tk.W)
        
        # Detected plates display
        plates_frame = ttk.LabelFrame(right_panel, text="🔤 Detected License Plates", padding="10")
        plates_frame.pack(fill=tk.BOTH, expand=True)
        
        # Create treeview for detected plates
        columns = ('Time', 'Plate Text', 'Confidence', 'OCR Confidence')
        self.plates_tree = ttk.Treeview(plates_frame, columns=columns, show='headings', height=8)
        
        # Define column headings and widths
        self.plates_tree.heading('Time', text='🕐 Time')
        self.plates_tree.heading('Plate Text', text='🔤 Plate Text')
        self.plates_tree.heading('Confidence', text='🎯 Det. Conf.')
        self.plates_tree.heading('OCR Confidence', text='📖 OCR Conf.')
        
        self.plates_tree.column('Time', width=80)
        self.plates_tree.column('Plate Text', width=120)
        self.plates_tree.column('Confidence', width=80)
        self.plates_tree.column('OCR Confidence', width=80)
        
        # Add scrollbar
        scrollbar = ttk.Scrollbar(plates_frame, orient=tk.VERTICAL, command=self.plates_tree.yview)
        self.plates_tree.configure(yscrollcommand=scrollbar.set)
        
        self.plates_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # Status bar
        self.status_var = tk.StringVar(value="📱 Ready - Click 'Start Detection' to begin")
        status_bar = ttk.Label(main_frame, textvariable=self.status_var, 
                              relief=tk.SUNKEN, anchor=tk.W, font=('Arial', 9))
        status_bar.pack(fill=tk.X, pady=(10, 0))
        
        # Update model info
        if self.model:
            model_info = f"Model: Custom ({len(self.model.names)} classes)"
        else:
            model_info = "Model: Not loaded"
        self.model_var.set(model_info)
    
    def update_confidence(self, value):
        """Update confidence threshold"""
        self.confidence_threshold = float(value)
        self.confidence_label.config(text=f"Value: {self.confidence_threshold:.2f}")
    
    def toggle_ocr(self):
        """Toggle OCR functionality"""
        self.ocr_enabled = self.ocr_var.get()
        status = "Enabled" if self.ocr_enabled else "Disabled"
        self.ocr_status_var.set(f"OCR: {status}")
        print(f"🔤 OCR {status}")
    
    def preprocess_for_ocr(self, image):
        """Preprocess license plate image for better OCR"""
        try:
            # Convert to grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            # Apply Gaussian blur to reduce noise
            blurred = cv2.GaussianBlur(gray, (3, 3), 0)
            
            # Apply adaptive thresholding
            thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                         cv2.THRESH_BINARY, 11, 2)
            
            # Morphological operations to clean up
            kernel = np.ones((2, 2), np.uint8)
            cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
            
            # Resize for better OCR (make it larger)
            height, width = cleaned.shape
            if height < 50:
                scale_factor = 50 / height
                new_width = int(width * scale_factor)
                cleaned = cv2.resize(cleaned, (new_width, 50), interpolation=cv2.INTER_CUBIC)
            
            return cleaned
        except Exception as e:
            print(f"❌ OCR preprocessing error: {e}")
            return image
    
    def clean_plate_text(self, text):
        """Clean and format detected plate text"""
        if not text:
            return ""
        
        # Remove special characters and keep only alphanumeric
        cleaned = re.sub(r'[^A-Za-z0-9]', '', text.upper())
        
        # Vietnamese license plate patterns
        patterns = [
            r'^(\d{2}[A-Z]\d{4,5})$',  # 12A1234 or 12A12345
            r'^(\d{2}[A-Z]{2}\d{3,4})$',  # 12AB123 or 12AB1234
            r'^(\d{2}[A-Z]\d{3}\.\d{2})$',  # 12A123.45
            r'^([A-Z]{2}\d{4,5})$',  # AB1234 or AB12345
        ]
        
        # Try to match Vietnamese patterns
        for pattern in patterns:
            match = re.search(pattern, cleaned)
            if match:
                return match.group(1)
        
        # If no pattern matches, return cleaned text if it's reasonable length
        if 4 <= len(cleaned) <= 10:
            return cleaned
        
        return text.strip() if text.strip() else "UNKNOWN"
    
    def perform_ocr(self, plate_image):
        """Perform OCR on license plate image"""
        if not self.ocr_reader or not self.ocr_enabled:
            return "OCR Disabled", 0.0
        
        try:
            # Preprocess image for better OCR
            processed_image = self.preprocess_for_ocr(plate_image)
            
            # Perform OCR
            results = self.ocr_reader.readtext(processed_image, detail=1, paragraph=False)
            
            if not results:
                return "NO TEXT", 0.0
            
            # Find the best result (highest confidence)
            best_text = ""
            best_confidence = 0.0
            
            for (bbox, text, confidence) in results:
                if confidence > best_confidence and len(text.strip()) > 0:
                    best_text = text
                    best_confidence = confidence
            
            if best_text:
                cleaned_text = self.clean_plate_text(best_text)
                return cleaned_text, best_confidence
            else:
                return "NO TEXT", 0.0
                
        except Exception as e:
            print(f"❌ OCR error: {e}")
            return "OCR ERROR", 0.0
    
    def add_detected_plate(self, plate_text, det_confidence, ocr_confidence):
        """Add detected plate to the history"""
        current_time = datetime.now().strftime("%H:%M:%S")
        
        # Check if this plate was recently detected (avoid duplicates)
        recent_plates = [item for item in self.detected_plates if item['time'] > time.time() - 2]
        for plate in recent_plates:
            if plate['text'] == plate_text:
                return  # Skip duplicate
        
        # Add to internal list
        plate_data = {
            'time': time.time(),
            'timestamp': current_time,
            'text': plate_text,
            'det_confidence': det_confidence,
            'ocr_confidence': ocr_confidence
        }
        self.detected_plates.append(plate_data)
        
        # Add to treeview
        self.plates_tree.insert('', 0, values=(
            current_time,
            plate_text,
            f"{det_confidence:.2f}",
            f"{ocr_confidence:.2f}" if ocr_confidence > 0 else "N/A"
        ))
        
        # Keep only last 50 entries
        if len(self.detected_plates) > 50:
            self.detected_plates = self.detected_plates[-50:]
            
        # Remove old entries from treeview
        children = self.plates_tree.get_children()
        if len(children) > 50:
            for item in children[50:]:
                self.plates_tree.delete(item)
        
        print(f"🔤 Detected plate: {plate_text} (Det: {det_confidence:.2f}, OCR: {ocr_confidence:.2f})")
    
    def save_detected_plates(self):
        """Save detected plates to file"""
        if not self.detected_plates:
            messagebox.showwarning("No Data", "No license plates detected yet.")
            return
        
        try:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"detected_plates_{timestamp}.txt"
            
            with open(filename, 'w', encoding='utf-8') as f:
                f.write("Red License Plate Detection Results\n")
                f.write("="*50 + "\n")
                f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"Total detections: {len(self.detected_plates)}\n\n")
                
                for i, plate in enumerate(self.detected_plates, 1):
                    f.write(f"{i:3d}. Time: {plate['timestamp']} | ")
                    f.write(f"Plate: {plate['text']:12s} | ")
                    f.write(f"Det.Conf: {plate['det_confidence']:.3f} | ")
                    f.write(f"OCR.Conf: {plate['ocr_confidence']:.3f}\n")
            
            messagebox.showinfo("Saved", f"Detected plates saved to:\n{filename}")
            self.status_var.set(f"💾 Plates saved to {filename}")
            
        except Exception as e:
            messagebox.showerror("Save Error", f"Cannot save file: {e}")
    
    def clear_history(self):
        """Clear detection history"""
        self.detected_plates.clear()
        for item in self.plates_tree.get_children():
            self.plates_tree.delete(item)
        self.status_var.set("🗑️ Detection history cleared")
        print("🗑️ Detection history cleared")
    
    def initialize_webcam(self):
        """Initialize webcam connection"""
        try:
            # Try different camera indices
            for camera_idx in [0, 1, 2]:
                print(f"🔄 Trying camera index {camera_idx}...")
                test_cap = cv2.VideoCapture(camera_idx)
                
                if test_cap.isOpened():
                    # Test if we can read a frame
                    ret, frame = test_cap.read()
                    if ret:
                        self.cap = test_cap
                        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
                        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
                        self.cap.set(cv2.CAP_PROP_FPS, 30)
                        
                        width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                        height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        
                        print(f"✅ Camera {camera_idx} initialized: {width}x{height}")
                        self.camera_var.set(f"Camera: {camera_idx} ({width}x{height})")
                        self.status_var.set(f"📹 Camera {camera_idx} ready - Click 'Start Detection'")
                        return
                    else:
                        test_cap.release()
                else:
                    test_cap.release()
            
            # No camera found
            self.cap = None
            self.camera_var.set("Camera: Not found")
            self.status_var.set("❌ No camera detected")
            messagebox.showwarning("Camera Warning", 
                                 "No camera detected. Please connect a camera and click 'Reset Camera'")
            
        except Exception as e:
            print(f"❌ Camera initialization error: {e}")
            self.cap = None
            self.camera_var.set("Camera: Error")
            self.status_var.set("❌ Camera initialization failed")
    
    def toggle_detection(self):
        """Start or stop detection"""
        if not self.is_running:
            if self.cap is None:
                self.reset_camera()
                if self.cap is None:
                    messagebox.showerror("Error", "Cannot start detection: No camera available")
                    return
            
            if self.model is None:
                messagebox.showerror("Error", "Cannot start detection: No model loaded")
                return
            
            self.is_running = True
            self.start_button.config(text="⏹️ Stop Detection")
            self.status_var.set("🔄 Detection running...")
            
            # Start detection thread
            self.detection_thread = threading.Thread(target=self.detection_loop, daemon=True)
            self.detection_thread.start()
            
            # Start display update
            self.update_display()
        else:
            self.stop_detection()
    
    def stop_detection(self):
        """Stop detection"""
        self.is_running = False
        self.start_button.config(text="▶️ Start Detection")
        self.status_var.set("⏹️ Detection stopped")
        self.detection_var.set("Detections: 0")
        self.fps_var.set("FPS: 0.0")
    
    def detection_loop(self):
        """Main detection loop with OCR"""
        fps_counter = 0
        fps_start_time = time.time()
        last_ocr_time = 0
        
        while self.is_running and self.cap is not None:
            try:
                ret, frame = self.cap.read()
                if not ret:
                    print("❌ Cannot read frame from camera")
                    break
                
                current_time = time.time()
                
                # Run detection
                if self.model:
                    results = self.model(frame, conf=self.confidence_threshold, verbose=False)
                    
                    detection_count = 0
                    for r in results:
                        if len(r.boxes) > 0:
                            detection_count = len(r.boxes)
                            for box in r.boxes:
                                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                                conf = box.conf[0].cpu().numpy()
                                
                                # Draw rectangle
                                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 3)
                                
                                # Extract license plate region for OCR
                                plate_region = frame[int(y1):int(y2), int(x1):int(x2)]
                                
                                # Perform OCR (limit frequency to avoid slowdown)
                                plate_text = "DETECTING..."
                                ocr_conf = 0.0
                                
                                if (self.ocr_enabled and self.ocr_reader and 
                                    current_time - last_ocr_time > 1.0 and  # OCR every 1 second
                                    plate_region.shape[0] > 20 and plate_region.shape[1] > 40):  # Minimum size
                                    
                                    plate_text, ocr_conf = self.perform_ocr(plate_region)
                                    last_ocr_time = current_time
                                    
                                    # Add to detection history if OCR successful
                                    if plate_text != "DETECTING..." and plate_text != "NO TEXT" and ocr_conf > 0.3:
                                        self.add_detected_plate(plate_text, conf, ocr_conf)
                                
                                # Draw label with OCR result
                                label = f"{plate_text} ({conf:.2f})"
                                label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
                                
                                # Draw label background
                                cv2.rectangle(frame, (int(x1), int(y1-35)), 
                                            (int(x1) + label_size[0] + 10, int(y1)), (0, 255, 0), -1)
                                
                                # Draw label text
                                cv2.putText(frame, label, (int(x1) + 5, int(y1-10)), 
                                          cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
                    
                    self.detection_count = detection_count
                
                # Calculate FPS
                fps_counter += 1
                if fps_counter % 30 == 0:
                    current_time = time.time()
                    elapsed = current_time - fps_start_time
                    if elapsed > 0:
                        self.fps = 30 / elapsed
                    fps_start_time = current_time
                    fps_counter = 0
                
                # Put frame in queue for display
                try:
                    self.frame_queue.put_nowait(frame)
                except queue.Full:
                    try:
                        self.frame_queue.get_nowait()  # Remove old frame
                        self.frame_queue.put_nowait(frame)
                    except queue.Empty:
                        pass
                
                time.sleep(0.03)  # ~30 FPS
                
            except Exception as e:
                print(f"Detection loop error: {e}")
                break
        
        if self.is_running:
            self.root.after(0, self.stop_detection)
    
    def update_display(self):
        """Update the video display"""
        if self.is_running:
            try:
                frame = self.frame_queue.get_nowait()
                self.current_frame = frame.copy()
                
                # Convert frame to display format
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_pil = Image.fromarray(frame_rgb)
                
                # Resize frame to fit display
                display_width = 640
                display_height = 480
                frame_pil = frame_pil.resize((display_width, display_height), Image.Resampling.LANCZOS)
                
                # Convert to PhotoImage
                frame_tk = ImageTk.PhotoImage(frame_pil)
                
                # Update label
                self.video_label.config(image=frame_tk, text="")
                self.video_label.image = frame_tk  # Keep a reference
                
                # Update info displays
                self.detection_var.set(f"Detections: {self.detection_count}")
                self.fps_var.set(f"FPS: {self.fps:.1f}")
                
            except queue.Empty:
                pass
            except Exception as e:
                print(f"Display update error: {e}")
            
            # Schedule next update
            self.root.after(30, self.update_display)
    
    def take_screenshot(self):
        """Take a screenshot of current frame"""
        if hasattr(self, 'current_frame') and self.current_frame is not None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"license_plate_ocr_screenshot_{timestamp}.jpg"
            
            try:
                cv2.imwrite(filename, self.current_frame)
                messagebox.showinfo("Screenshot Saved", f"Screenshot saved as:\n{filename}")
                self.status_var.set(f"📸 Screenshot saved: {filename}")
                print(f"📸 Screenshot saved: {filename}")
            except Exception as e:
                messagebox.showerror("Screenshot Error", f"Failed to save screenshot: {e}")
        else:
            messagebox.showwarning("No Frame", "No frame available for screenshot.\nStart detection first.")
    
    def reset_camera(self):
        """Reset camera connection"""
        self.stop_detection()
        
        if self.cap:
            self.cap.release()
            self.cap = None
        
        self.status_var.set("🔄 Resetting camera...")
        self.root.update()
        
        time.sleep(1)  # Wait a moment
        self.initialize_webcam()
    
    def quit_app(self):
        """Clean up and quit application"""
        self.stop_detection()
        
        if self.cap:
            self.cap.release()
        
        self.root.quit()
        self.root.destroy()
        print("👋 Application closed")

def main():
    """Main function to run the OCR application"""
    print("🚀 Starting License Plate Detection & OCR GUI...")
    print("📦 Required packages: easyocr, ultralytics, opencv-python, pillow")
    
    # Check if EasyOCR is installed
    try:
        import easyocr
        print("✅ EasyOCR is available")
    except ImportError:
        print("❌ EasyOCR not found. Installing...")
        try:
            import subprocess
            subprocess.check_call(["pip", "install", "easyocr"])
            print("✅ EasyOCR installed successfully")
        except Exception as e:
            print(f"❌ Cannot install EasyOCR: {e}")
            print("💡 Please run: pip install easyocr")
    
    # Create and configure root window
    root = tk.Tk()
    
    # Set up styles
    style = ttk.Style()
    style.theme_use('clam')  # Use a modern theme
    
    # Create and run application
    app = LicensePlateOCRApp(root)
    
    try:
        root.mainloop()
    except KeyboardInterrupt:
        print("\n⏹️ Application interrupted by user")
    except Exception as e:
        print(f"❌ Application error: {e}")
    finally:
        if hasattr(app, 'cap') and app.cap:
            app.cap.release()
        print("🔚 Application terminated")

if __name__ == "__main__":
    main()

🚀 Starting License Plate Detection & OCR GUI...
📦 Required packages: easyocr, ultralytics, opencv-python, pillow
✅ EasyOCR is available
✅ Loading model from: runs/detect/red_plate_memory_opt/weights/best.pt


Using CPU. Note: This module is much faster with a GPU.


📊 Model classes: {0: 'License_Plate'}
🔤 Loading OCR reader...
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete✅ OCR reader loaded successfully
🔄 Trying camera index 0...
✅ Camera 0 initialized: 640x480
🔤 Detected plate: 381 (Det: 0.34, OCR: 0.44)
🔤 Detected plate: 5347 (Det: 0.44, OCR: 0.57)
🔤 Detected plate: 53 (Det: 0.37, OCR: 1.00)
🔤 Detected plate: 53070 (Det: 0.32, OCR: 0.42)
🔤 Detected plate: 53274 (Det: 0.32, OCR: 0.53)
🔤 Detected plate: JBHT (Det: 0.30, OCR: 0.53)
🗑️ Detection history cleared
🗑️ Detection history cleared
🔤 Detected plate: BH (Det: 0.12, OCR: 0.99)
🔤 Detected plate: 5324 (Det: 0.34, OCR: 0.60)
🔤 Detected plate: BH (Det: 0.37, OCR: 0.99)
🔤 Detected plate: 5324 (Det: 0.38, OCR: 0.87)
🔤 Detected plate: BH (Det: 0.48, OCR: 0.99)
🔤 Detected plate: 5324 (Det: 0.67, OCR: 0.56)
🗑️ Detection history cleared
🔤 Detected plate: BH5324 (Det: 0.70, OCR: 0.78)
🔤 Detected plate: B4P53224 (Det: 0.65, OCR: 0.43)
🔤 Detected plate: BH (Det: 0.10, OCR: 0.99)
🔤 Detected plate: I537 (Det: 0.10, OCR: 0.77)
🔤 Detected